# Harmonization of MIDRC Imaging Study Descriptions using the LOINC Playbook
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

November 2022

---
This Jupyter notebook demonstrates how to map the imaging_study node's `study_description` and `modality` properties to a `loinc_code` by utilizing the MIDRC DQH committee's [LOINC mapping table in GitHub](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out). Once the imaging study has been mapped to a LOINC code, the code can be used to derive other LOINC properties. The following [LOINC properties](https://github.com/uc-cdis/midrc_dictionary/blob/deae581f0fb8b9ae5add1458d7882e189ba97af6/gdcdictionary/schemas/imaging_study.yaml#L77) are on the `imaging_study` node of the [MIDRC data dictionary](https://data.midrc.org/dd):
* loinc_code
* loinc_long_common_name
* loinc_method
* loinc_system
* loinc_contrast

In [1]:
# Import Python Packages and scripts
import pandas as pd
import numpy as np
from pathlib import Path
import sys, os, copy, datetime, shutil


import gen3
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


In [2]:
# download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
loinc_dir = "/Users/christopher/Documents/Notes/MIDRC/LOINC" #set your working dir
os.chdir(loinc_dir)
os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O expansion.py")
from expansion import Gen3Expansion


In [3]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
sapi = 'https://staging.midrc.org'
scred = '/Users/christopher/Downloads/midrc-staging-credentials.json'
sauth = Gen3Auth(sapi, refresh_file=scred) # authentication class
ssub = Gen3Submission(sapi, sauth) # submission class
squery = Gen3Query(sauth) # query class
sexp = Gen3Expansion(sapi,sauth,ssub) # class with some custom scripts
sexp.get_project_ids()


Getting all project_ids you have access to in the data commons.
['Open-A1', 'Open-A1_PETAL_REDCORAL', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD']


/Users/christopher/Documents/Notes/MIDRC/LOINC/expansion.py:213: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(res["data"]["project"])


['Open-A1',
 'Open-A1_PETAL_REDCORAL',
 'Open-R1',
 'TCIA-COVID-19-AR',
 'TCIA-COVID-19-NY-SBU',
 'TCIA-COVID-19_CT_Images',
 'TCIA-RICORD']

## Prepare the mapping and filter tables
---
Get the mapping table and the filtering attributes tables from the [MIDRC GitHub repository](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out) and reformat them to prepare for the LOINC mapping.

Prior to this, you will need to clone the repo using `git clone git@github.com:MIDRC/midrc_dicom_harmonization.git`


In [4]:
## Set the directory to your copy of the GitHub repo and pull main branch to pull latest updates from GitHub
git_dir = "/Users/christopher/Documents/GitHub/MIDRC/midrc_dicom_harmonization/"
os.chdir(git_dir)
os.system("git checkout main")
os.system("git pull origin main")
os.chdir(loinc_dir)


In [5]:
## Create the output/working directory
now = datetime.datetime.now()
today = "{}-{}-{}".format(now.year, now.month, now.day)

results_dir = "{}/results_{}".format(loinc_dir,today)
Path(results_dir).mkdir(parents=True, exist_ok=True)

In [19]:
mapping_file = "{}/out/StudyDescription_mapping_table.csv".format(git_dir)
mapping = pd.read_csv(mapping_file,dtype=str) #['Modality', 'StudyDescription', 'LOINC code', 'L-Long Common Name']

shutil.copy2(mapping_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

mapping.rename(columns={"StudyDescription":"study_description","Modality":"study_modality","LOINC code":"loinc_code","L-Long Common Name":"loinc_long_common_name"},inplace=True)
mapping.drop_duplicates(inplace=True)
mapping['study_description'] = mapping.apply(lambda row: row['study_description'].casefold(),axis=1)
mapping['study_modality'] = mapping.apply(lambda row: row['study_modality'].casefold(),axis=1)

## Change any "(blank)" values for study_description to "[blank]" to match the mapping table
mapping.replace({"study_description":{
    '(blank)':'[blank]'}},
    inplace=True)

## remove any leading/trailing whitespaces in the codes/descriptions
mapping['loinc_code'] = mapping['loinc_code'].str.strip()
mapping['loinc_long_common_name'] = mapping['loinc_long_common_name'].str.strip()
mapping['study_description'] = mapping['study_description'].str.strip()
mapping['study_modality'] = mapping['study_modality'].str.strip()

## Summarize the mapping table
modalities = list(set(mapping['study_modality']))
descriptions = list(set(mapping['study_description']))
mapping_codes = list(set(mapping['loinc_code']))
mapping_names = list(set(mapping['loinc_long_common_name']))

display(mapping)
print("study_modality values in mapping table: {}".format(modalities))
print("Number of unique LOINC codes in mapping table: {}".format(len(mapping_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(mapping_names)))


,study_modality,study_description,loinc_code,loinc_long_common_name
0,ct,[blank],25045-6,CT Unspecified body region
1,ct,pet ct fdg imag skull to thigh,81555-5,PET+CT Guidance for localization of tumor of W...
2,ct,chest pe(adult),79077-4,CTA Pulmonary arteries for pulmonary embolus W...
3,ct,ct chest pulmonary angio with iv con,79077-4,CTA Pulmonary arteries for pulmonary embolus W...
4,ct,ct chest pulmonary embolism (ctpe),79077-4,CTA Pulmonary arteries for pulmonary embolus W...
...,...,...,...,...
301,mg,[blank],36625-2,MG Breast Views
302,nm,[blank],49118-3,NM Unspecified body region Views
303,st,[blank],43526-3,SPECT Unspecified body region
304,[blank],bronchoscopy,18744-3,Bronchoscopy study


study_modality values in mapping table: ['nm', 'pt, ctpt', 'mr', 'ct', 'rf', 'us', 'mg', 'cr, dx', '[blank]', 'st']
Number of unique LOINC codes in mapping table: 83
Number of unique LOINC Long Common Names in mapping table: 83


In [21]:
filters_file = "{}/out/StudyDescription_filtering_attributes.csv".format(git_dir)
filters = pd.read_csv(filters_file,dtype=str) #['LOINC code', 'L-Long Common Name', 'L-Method', 'L-System', 'Rad.Timing']

shutil.copy2(filters_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

filters.rename(columns= {
    'LOINC code':'loinc_code',
    'L-Long Common Name':'loinc_long_common_name',
    'L-Method':'loinc_method',
    'Rad.Timing':'loinc_contrast',
    'MIDRC-System':'loinc_system'
    },
    inplace=True,
    errors='ignore'
)
filters.drop(columns='L-System',inplace=True,errors='ignore')

## remove any leading/trailing whitespaces
filters['loinc_code'] = filters['loinc_code'].str.strip()
filters['loinc_long_common_name'] = filters['loinc_long_common_name'].str.strip()

filter_codes = list(set(filters['loinc_code']))
filter_names = list(set(filters['loinc_long_common_name']))
print("Number of unique LOINC codes in filters table: {}".format(len(filter_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(filter_names)))


missing_codes = set(mapping_codes).difference(set(filter_codes))
missing_names = set(mapping_names).difference(set(filter_names))

print("Number of LOINC codes in mapping table missing from filters table: {}\n\t{}".format(len(missing_codes),missing_codes))
print("Number of LOINC names in mapping table missing from filters table: {}\n\t{}".format(len(missing_names),missing_names))

display(filters)


Number of unique LOINC codes in filters table: 82
Number of unique LOINC Long Common Names in mapping table: 82
Number of LOINC codes in mapping table missing from filters table: 1
	{'18744-3'}
Number of LOINC names in mapping table missing from filters table: 1
	{'Bronchoscopy study'}


,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,35889-5,RF Guidance for bronchoscopy of Chest,RF,NaN,Chest
1,36813-4,CT Abdomen and Pelvis W contrast IV,CT,W,Abdomen
2,42274-1,CT Abdomen and Pelvis WO and W contrast IV,CT,WO & W,Abdomen+Pelvis
3,36952-0,CT Abdomen and Pelvis WO contrast,CT,WO,Abdomen+Pelvis
4,79103-8,CT Abdomen W contrast IV,CT,W,Abdomen
...,...,...,...,...,...
77,83017-4,XR Chest View and Abdomen Supine and Upright,XR,NaN,Chest && Abdomen
78,30745-4,XR Chest Views,XR,NaN,Chest
79,24899-7,XR Ribs Views,XR,NaN,Chest>Ribs
80,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified


## Get the imaging_study information from MIDRC Staging
---
* Use the [`Gen3Expansion.get_node_tsvs()` SDK function](https://github.com/cgmeyer/gen3sdk-python/blob/389e3945482439ace6e4536e6d0e35c6e48de9c9/expansion/expansion.py#L219) to get all the imaging_studies in MIDRC Staging (staging.midrc.org).

* The function `exp.get_node_tsvs()` will return a master dataframe containing all the imaging_study data from the projects specified.

### In this version of the notebook, we're only getting three projects: 
* Open-R1 (data from RSNA)
* Open-A1 (data from ACR)
* Open-A1_PETAL_REDCORAL (PETAL data from ACR)


In [22]:
os.chdir(loinc_dir)
projects = ['Open-R1','Open-A1','Open-A1_PETAL_REDCORAL']
#st = sexp.get_node_tsvs(node='imaging_study', overwrite=False, projects=projects,outdir=results_dir)
st = sexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=projects,outdir=results_dir)



Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/imaging_study_tsvs/Open-R1_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/imaging_study_tsvs/Open-R1_imaging_study.tsv has 46253 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/imaging_study_tsvs/Open-A1_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/imaging_study_tsvs/Open-A1_imaging_study.tsv has 46083 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/imaging_study_tsvs/Open-A1_PETAL_REDCORAL_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/imaging_study_tsvs/Open-A1_PETAL_REDCORAL_imaging_study.tsv has 3170 records.
length of all dfs: 95506
Master node TSV with 95506 total records written to master_imaging_study.tsv.


In [23]:
dupes = st.loc[st.duplicated(subset='submitter_id',keep=False)].sort_values(by='submitter_id')
print("There are {} duplicate submitter_ids in the list of imaging studies.".format(len(dupes)))
if len(dupes) > 0:
    display(dupes)



There are 0 duplicate submitter_ids in the list of imaging studies.


## Reformat the imaging_study data to prepare for mapping
---
* **NaN values**: In the mapping table, a `study_description` or `study_modality` with a value of `null` or `NaN` is listed as `[blank]`.
* **Case-insensitivity**: we should ignore capitalization of `study_modality` and `study_description` strings when we do the look-up. We'll use `str.casefold()` to ignore case for matching.
* **CR/DX Modality**: in the mapping table, all versions of "CR" and "DX" modalities are denoted "CR, DX"; so, we need to change all occurrences of other spellings to match that exactly.
* **Duplicated study UIDs**: Identify any duplicated study UIDs and remove them from studies.


In [24]:
## Make a copy of the master imaging_study table with only data relevant to LOINC mapping, and change NaN to "[blank]"
studies = copy.deepcopy(st[['submitter_id','study_description','study_modality']])
#studies.index = studies['submitter_id']
studies['study_description'].fillna("[blank]", inplace=True)
studies['study_modality'].fillna("[blank]", inplace=True)

## Make the mapping case-insensitive
studies['study_description'] = studies['study_description'].str.casefold()
studies['study_modality'] = studies['study_modality'].str.casefold()

## remove any leading/trailing whitespaces
studies['study_description'] = studies['study_description'].str.strip()
studies['study_modality'] = studies['study_modality'].str.strip()


## Fix any CR/DX study_modality to all be "cr, dx" to match mapping table
studies.replace({"study_modality":{
    'cr':'cr, dx',
    'dx':'cr, dx',
    'cr,dx':'cr, dx',
    'dx,cr':'cr, dx',
    'dx, cr':'cr, dx',
    'crdx':'cr, dx',
    'dxcr':'cr, dx'}},
    inplace=True)

## Fix any CT/PT study_modality to all be "pt, ct" to match mapping table
studies.replace({"study_modality":{
    'pt':'pt, ctpt',
    'ctpt':'pt, ctpt',
    'ptct':'pt, ctpt',
    'ct, pt':'pt, ctpt',
    'pt, ct':'pt, ctpt',
    'pt,ct':'pt, ctpt',
    'ct,pt':'pt, ctpt'}},
    inplace=True)

## Change any "<NONE>" values for study_description to "[blank]" to match the mapping table
studies.replace({"study_description":{
    '<none>':'[blank]'}},
    inplace=True)

## Identify and drop duplicates
dupes = studies.loc[studies.duplicated(keep=False)]
studies = studies.drop_duplicates()
display(studies)



,submitter_id,study_description,study_modality
0,1.2.826.0.1.3680043.10.474.419639.104706049142...,xr chest 1 view ap,"cr, dx"
1,1.2.826.0.1.3680043.10.474.419639.254966223507...,ct chest pulmonary embolism (ctpe),ct
2,1.2.826.0.1.3680043.10.474.419639.407911005817...,xr chest 1 view ap,"cr, dx"
3,1.2.826.0.1.3680043.10.474.514382.1806498,"xr chest 1 vw, frontal","cr, dx"
4,1.2.826.0.1.3680043.10.474.514382.1159040,cta chest pulmonary angiography w contrast,ct
...,...,...,...
95501,2.16.840.1.114274.1818.55861569088646803211242...,[blank],"cr, dx"
95502,2.16.840.1.114274.1818.53048606769276580282045...,"xr chest 2 view (pa,lat)","cr, dx"
95503,2.16.840.1.114274.1818.50393399609728206284180...,xr chest 1 view,"cr, dx"
95504,2.16.840.1.114274.1818.51197741541012424551144...,xray chest one view,"cr, dx"


In [25]:
## Check modalities for presence in the mapping table and frequency in MIDRC data:
study_modalities = list(set(studies.study_modality))
print("There are the following values of study_modality in the MIDRC imaging_study data: \n{}".format(study_modalities))

missing_modalities = set(study_modalities).difference(modalities)

mfreq = {}
for mod in missing_modalities:
    freq = len(studies.loc[studies['study_modality']==mod])
    mfreq[mod] = freq

print("\nThese modalities are in the study data but not in the mapping table: \n{}".format(mfreq))

There are the following values of study_modality in the MIDRC imaging_study data: 
['pt, ctpt', 'mr', 'pr', 'ct', 'dr', 'xa', 'rf', 'mg', 'us', 'cr, dx']

These modalities are in the study data but not in the mapping table: 
{'pr': 1, 'dr': 2, 'xa': 1}


## Do mapping using pandas pd.merge(): 
---

1) Merge `loinc_code` in `mapping` dataframe into the `studies` dataframe on the combination of `study_description` and `study_modality`.

2) Merge the `loinc_long_common_name`, `loinc_method`, `loinc_contrast`, and `loinc_system` in `filters` dataframe into the `studies` dataframe on `loinc_code`.



In [26]:
sdf = studies.merge(mapping,on=['study_modality','study_description'],how='left').drop_duplicates()
sdf = sdf.merge(filters,on=['loinc_code','loinc_long_common_name'],how='left')
sdf.drop(columns=['study_description','study_modality'],inplace=True,errors='ignore')
sdf

,submitter_id,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,1.2.826.0.1.3680043.10.474.419639.104706049142...,36572-6,XR Chest AP,XR,NaN,Chest
1,1.2.826.0.1.3680043.10.474.419639.254966223507...,79077-4,CTA Pulmonary arteries for pulmonary embolus W...,CT.angio,W,Chest
2,1.2.826.0.1.3680043.10.474.419639.407911005817...,36572-6,XR Chest AP,XR,NaN,Chest
3,1.2.826.0.1.3680043.10.474.514382.1806498,36554-4,XR Chest Single view,XR,NaN,Chest
4,1.2.826.0.1.3680043.10.474.514382.1159040,79077-4,CTA Pulmonary arteries for pulmonary embolus W...,CT.angio,W,Chest
...,...,...,...,...,...,...
95501,2.16.840.1.114274.1818.55861569088646803211242...,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified
95502,2.16.840.1.114274.1818.53048606769276580282045...,NaN,NaN,NaN,NaN,NaN
95503,2.16.840.1.114274.1818.50393399609728206284180...,36554-4,XR Chest Single view,XR,NaN,Chest
95504,2.16.840.1.114274.1818.51197741541012424551144...,36554-4,XR Chest Single view,XR,NaN,Chest


In [27]:
## Pull in the original study_modality and study_description (non-lowercase) and rearrange the columns
cols = ['type','project_id','submitter_id','cases.submitter_id','study_modality','study_description','loinc_code','loinc_long_common_name','loinc_method','loinc_contrast','loinc_system']
sdf = sdf.merge(st[['type','project_id','submitter_id','cases.submitter_id','study_description','study_modality']], on='submitter_id',how='left')[cols]
sdf


,type,project_id,submitter_id,cases.submitter_id,study_modality,study_description,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,imaging_study,Open-R1,1.2.826.0.1.3680043.10.474.419639.104706049142...,419639-008711,CR,XR CHEST 1 VIEW AP,36572-6,XR Chest AP,XR,NaN,Chest
1,imaging_study,Open-R1,1.2.826.0.1.3680043.10.474.419639.254966223507...,419639-003500,CT,CT CHEST PULMONARY EMBOLISM (CTPE),79077-4,CTA Pulmonary arteries for pulmonary embolus W...,CT.angio,W,Chest
2,imaging_study,Open-R1,1.2.826.0.1.3680043.10.474.419639.407911005817...,419639-007684,DX,XR CHEST 1 VIEW AP,36572-6,XR Chest AP,XR,NaN,Chest
3,imaging_study,Open-R1,1.2.826.0.1.3680043.10.474.514382.1806498,514382-016191,DX,"XR CHEST 1 VW, FRONTAL",36554-4,XR Chest Single view,XR,NaN,Chest
4,imaging_study,Open-R1,1.2.826.0.1.3680043.10.474.514382.1159040,514382-010684,CT,CTA CHEST PULMONARY ANGIOGRAPHY W CONTRAST,79077-4,CTA Pulmonary arteries for pulmonary embolus W...,CT.angio,W,Chest
...,...,...,...,...,...,...,...,...,...,...,...
95501,imaging_study,Open-A1_PETAL_REDCORAL,2.16.840.1.114274.1818.55861569088646803211242...,C03-00804,DX,NaN,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified
95502,imaging_study,Open-A1_PETAL_REDCORAL,2.16.840.1.114274.1818.53048606769276580282045...,D01-01663,CR,"XR CHEST 2 VIEW (PA,LAT)",NaN,NaN,NaN,NaN,NaN
95503,imaging_study,Open-A1_PETAL_REDCORAL,2.16.840.1.114274.1818.50393399609728206284180...,D04-01057,DX,XR CHEST 1 VIEW,36554-4,XR Chest Single view,XR,NaN,Chest
95504,imaging_study,Open-A1_PETAL_REDCORAL,2.16.840.1.114274.1818.51197741541012424551144...,P01-01341,DX,XRAY CHEST ONE VIEW,36554-4,XR Chest Single view,XR,NaN,Chest


In [28]:
mdf = sdf.loc[~sdf['loinc_code'].isna()]
udf = sdf.loc[sdf['loinc_code'].isna()]
fdf = udf.groupby(["study_modality", "study_description"]).size().reset_index(name="Freq").sort_values(by='Freq',ascending=False)
print("Total Studies: {}, Mapped: {}, Unmapped: {}".format(len(sdf),len(mdf),len(udf)))

Total Studies: 95506, Mapped: 93769, Unmapped: 1737


In [29]:
## Save results to files
filename="{}/Open_LOINC_mapping_results_{}_{}.tsv".format(results_dir,len(sdf),today)
sdf.to_csv(filename, sep='\t', index=False)
print("All LOINC mapping results for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/Open_LOINC_mapped_{}_{}.tsv".format(results_dir,len(mdf),today)
mdf.to_csv(filename, sep='\t', index=False)
print("Successfully mapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/Open_LOINC_umapped_{}_{}.tsv".format(results_dir,len(udf),today)
udf.to_csv(filename, sep='\t', index=False)
print("Unmapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/Open_LOINC_umapped_frequencies_{}_{}.tsv".format(results_dir,len(fdf),today)
fdf.to_csv(filename, sep='\t', index=False)
print("Frequencies of unmapped imaging study description/modality combinations for projects {} saved to file: \n{}\n".format(projects,filename))


All LOINC mapping results for projects ['Open-R1', 'Open-A1', 'Open-A1_PETAL_REDCORAL'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/Open_LOINC_mapping_results_95506_2022-11-21.tsv

Successfully mapped imaging studies for projects ['Open-R1', 'Open-A1', 'Open-A1_PETAL_REDCORAL'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/Open_LOINC_mapped_93769_2022-11-21.tsv

Unmapped imaging studies for projects ['Open-R1', 'Open-A1', 'Open-A1_PETAL_REDCORAL'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/Open_LOINC_umapped_1737_2022-11-21.tsv

Frequencies of unmapped imaging study description/modality combinations for projects ['Open-R1', 'Open-A1', 'Open-A1_PETAL_REDCORAL'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-21/Open_LOINC_umapped_frequencies_816_2022-11-21.tsv



In [30]:
## Take a glance at frequency of the unmapped imaging study modality/description combinations
display(fdf)


,study_modality,study_description,Freq
694,DX,XR CHEST PA LATERAL W BOTH OBLIQUE PROJECTIONS,18
508,CT,JOINY CT CHEST WO,9
113,CR,XR Portable Chest 1 view,9
64,CR,Ribs R,8
149,CT,Abdomen^ABDOMEN_PELVIS_WO_ROUTINE (Adult),8
...,...,...,...
470,CT,JHN CAP WITH IP,1
184,CT,Abdomen^STONE_FULL (Adult),1
468,CT,JHN CAP W/ OP,1
467,CT,JHN CAP W/ IP,1


## Submit the data to staging
---
Use the Gen3SDK function `Gen3Submission.submit_file()` to update the LOINC properties for each of the newly mapped studies in `mdf`.

In [ ]:
# Submit the derived data to staging
projects = list(set(mdf['project_id']))
data = {}
for pid in projects:
    print("Submitting data to project '{}'.".format(pid))
    data[pid] = sexp.submit_df(df=mdf.loc[mdf["project_id"]==pid],project_id=pid, chunk_size=1000)
